<a href="https://colab.research.google.com/github/stardust114514/vits/blob/main/vits%E5%8D%95%E4%BA%BA%E6%A8%A1%E5%9E%8B%E8%AE%AD%E7%BB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Forked from https://github.com/CjangCjengh/vits/blob/main/vits.ipynb

**请注意该笔记本为训练单人模型，不包括合成语音，而且默认只保存一个checkpoint以节省空间。**

**默认每隔1000次迭代保存一次。在看到进度保存之前不要轻易退出，以免丢失进度。**

vits的配置较繁琐，简单配置参照[tacotron2笔记本](https://colab.research.google.com/drive/18fbCupSaQde-FtF2Z2Na-LP5BrukjNMs?usp=sharing)


训练多人模型需要自行对模型和配置文件进行修改。



In [14]:
#@title 准备 
# forked from https://www.endpointdev.com/blog/2015/01/getting-realtime-output-using-python/
import os
import subprocess
def run_command(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    try:
      process = subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      out, err = process.communicate()
    except:
      pass
    print_pipe(out)
    print_pipe(err)
    rc = process.poll()
    return rc

def run_command_by_line(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    with subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE) as process:
      while process.poll() is None:
        print_pipe(process.stdout.readline())
      [print_pipe(line) for line in process.stderr.readlines()]
    return

'''
Defines the set of symbols used in text input to the model.
'''

symbols_map = {
    "chinese_cleaners": {
        "_pad": '_',
        "_punctuation": '，。！？—…',
        "_letters": 'ㄅㄆㄇㄈㄉㄊㄋㄌㄍㄎㄏㄐㄑㄒㄓㄔㄕㄖㄗㄘㄙㄚㄛㄜㄝㄞㄟㄠㄡㄢㄣㄤㄥㄦㄧㄨㄩˉˊˇˋ˙ ',
    },
    "japanese_cleaners": {
        "_pad": '_',
        "_punctuation": ',.!?-',
        "_letters": 'AEINOQUabdefghijkmnoprstuvwyzʃʧ↓↑ '
    },
    "japanese_cleaners2": {
        "_pad": '_',
        "_punctuation": ',.!?-~…',
        "_letters": 'AEINOQUabdefghijkmnoprstuvwyzʃʧʦ↓↑ ',
    },
    "zh_ja_mixture_cleaners": {
        "_pad": '_',
        "_punctuation": ',.!?-~…~',
        "_letters": 'AEINOQUabdefghijklmnoprstuvwyzʃʧʦɯɹəɥ⁼ʰ`→↓↑',
    }
}


def get_symbols(specify_cleaners):
    if specify_cleaners not in symbols_map.keys():
        raise ValueError("不存在对应cleaners的symbols!")
    symbols = symbols_map[specify_cleaners]
    return [symbols["_pad"]] + list(symbols["_punctuation"]) + list(symbols["_letters"])

  

In [5]:
#@title 下载依赖库
#@markdown 取消勾选则不会节省空间
%cd /content
!git clone https://github.com/CjangCjengh/vits.git
%cd /content/vits
!pip install -r requirements.txt
!sudo apt-get install espeak -y
!sudo apt-get install p7zip-full p7zip-rar
!pip install demjson

/content
Cloning into 'vits'...
remote: Enumerating objects: 618, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 618 (delta 2), reused 8 (delta 2), pack-reused 607
Receiving objects: 100% (618/618), 31.30 MiB | 16.79 MiB/s, done.
Resolving deltas: 100% (292/292), done.
/content/vits
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 36.1 MB/s 
     |████████████████████████████████| 183 kB 76.4 MB/s 
     |████████████████████████████████| 11.6 MB 66.5 MB/s 
     |████████████████████████████████| 25.9 MB 89.2 MB/s 
     |████████████████████████████████| 6.8 MB 31.1 MB/s 
     |████████████████████████████████| 748.8 MB 19 kB/s 
     |████████████████████████████████| 5.9 MB 38.4 MB/s 
     |████████████████████████████████| 235 kB 65.2 MB/s 
     |████████████████████████████████| 1.5 MB 58.1 MB/s 
  Installing build dependen

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 5 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 k

In [6]:
#@title 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#@title 解压数据集
#@markdown 压缩包路径
import subprocess
dataset_path = "/content/drive/MyDrive/QWQ/wave"  #@param {type:"string"}
os.chdir('/content/vits')
run_command(["7z", "x", dataset_path])

NameError: ignored

In [10]:
%cd /content/vits
!7z x /content/drive/MyDrive/QWQ/hansermix.zip

/content/vits

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/QWQ/                                     1 file, 172896974 bytes (165 MiB)

Extracting archive: /content/drive/MyDrive/QWQ/hansermix.zip
--
Path = /content/drive/MyDrive/QWQ/hansermix.zip
Type = zip
Physical Size = 172896974

  0%      8% 96 - wave/keiko_0066.wav                              15% 196         24% 271 - wave/双花-251.wav                               29% 322 - wave/双花-298.wav                               37% 396 - wave/双花-365.wav

In [25]:
#@title 生成单人配置文件
# forked from https://github.com/CjangCjengh/vits/blob/main/configs/japanese_ss_base2.json
#@markdown 配置文件名称
json_filename = "model-single.json" #@param {type:"string"}
#@markdown 训练次数
hparams_epochs = 2000 #@param {type:"integer"}
#@markdown 单次迭代的文件数（建议在16以内）
hparams_batch_size = 16 #@param {type:"integer"}
#@markdown 训练集文件列表
hparams_training_files = "/content/vits/filelist_train.txt" #@param {type:"string"}
#@markdown 验证集文件列表
hparams_validation_files = "/content/vits/filelist_val.txt"#@param {type:"string"}
#@markdown 选择cleaner
hparams_cleaner =  "zh_ja_mixture_cleaners" #@param {type:"string"}
#@markdown 模型命名

#@markdown 生成的模型保存在/drive/MyDrive/你的模型命名/G.pth
hparams_speaker = "model-single" #@param {type:"string"}
hparams_symbols = "zh_ja_mixture_cleaners"
training_json = {
  "train": {
    "log_interval": 200,
    "eval_interval": 1000,
    "seed": 1234,
    "epochs": 10000,
    "learning_rate": 2e-4,
    "betas": [0.8, 0.99],
    "eps": 1e-9,
    "batch_size": 32,
    "fp16_run": True,
    "lr_decay": 0.999875,
    "segment_size": 8192,
    "init_lr_ratio": 1,
    "warmup_epochs": 0,
    "c_mel": 45,
    "c_kl": 1.0
  },
  "data": {
    "training_files":"filelists/filelist_train.txt.cleaned",
    "validation_files":"filelists/filelist_val.txt.cleaned",
    "text_cleaners":["zh_ja_mixture_cleaners"],
    "max_wav_value": 32768.0,
    "sampling_rate": 22050,
    "filter_length": 1024,
    "hop_length": 256,
    "win_length": 1024,
    "n_mel_channels": 80,
    "mel_fmin": 0.0,
    "mel_fmax": None,
    "add_blank": True,
    "n_speakers": 0,
    "cleaned_text": True
  },
  "model": {
    "inter_channels": 192,
    "hidden_channels": 192,
    "filter_channels": 768,
    "n_heads": 2,
    "n_layers": 6,
    "kernel_size": 3,
    "p_dropout": 0.1,
    "resblock": "1",
    "resblock_kernel_sizes": [3,7,11],
    "resblock_dilation_sizes": [[1,3,5], [1,3,5], [1,3,5]],
    "upsample_rates": [8,8,2,2],
    "upsample_initial_channel": 512,
    "upsample_kernel_sizes": [16,16,4,4],
    "n_layers_q": 3,
    "use_spectral_norm": False,
    "gin_channels": 256
  },
  "speakers": [hparams_speaker],
  "symbols":["_", ",", ".", "!", "?", "-", "~", "\u2026", "A", "E", "I", "N", "O", "Q", "U", "a", "b", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "r", "s", "t", "u", "v", "w", "y", "z", "\u0283", "\u02a7", "\u02a6", "\u026f", "\u0279", "\u0259", "\u0265", "\u207c", "\u02b0", "`", "\u2192", "\u2193", "\u2191", " "]
}

import demjson
os.chdir('/content/vits/configs')
training_json_text = demjson.encode(training_json)
with open(json_filename, "w") as file:
  file.write(training_json_text)

os.chdir('/content/vits/text')
with open("symbols.py", "w") as file:
  print("symbols = ", hparams_symbols, sep="", file=file)
os.chdir('/content/vits')



**目前支持的cleaner(和tacotron2版效果不同)**

cleaners from https://github.com/CjangCjengh/vits

1. japanese_cleaners 日语
2. korean_cleaners 韩语

In [46]:
#@title 预处理
os.chdir('/content/vits/monotonic_align')
!python setup.py build_ext --inplace
os.chdir('/content/vits')
run_command(["python", "preprocess.py", "--text_index", "1", "--text_cleaners", hparams_cleaner, "--filelists", hparams_training_files, hparams_validation_files])

running build_ext
START: /content/vits/filelist_train.txt
START: /content/vits/filelist_val.txt


0

In [50]:
#@title 训练
os.chdir('/content/vits')
run_command_by_line(["python", "train.py", "-c", "configs/{json}".format(json=json_filename), "-m", hparams_speaker])

INFO:model-single:{'data': {'add_blank': True, 'cleaned_text': True, 'filter_length': 1024, 'hop_length': 256, 'max_wav_value': 32768.0, 'mel_fmax': None, 'mel_fmin': 0.0, 'n_mel_channels': 80, 'n_speakers': 0, 'sampling_rate': 22050, 'text_cleaners': ['zh_ja_mixture_cleaners'], 'training_files': 'filelists/filelist_train.txt.cleaned', 'validation_files': 'filelists/filelist_val.txt.cleaned', 'win_length': 1024}, 'model': {'filter_channels': 768, 'gin_channels': 256, 'hidden_channels': 192, 'inter_channels': 192, 'kernel_size': 3, 'n_heads': 2, 'n_layers': 6, 'n_layers_q': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'resblock_kernel_sizes': [3, 7, 11], 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [16, 16, 4, 4], 'upsample_rates': [8, 8, 2, 2], 'use_spectral_norm': False}, 'speakers': ['model-single'], 'symbols': ['_', ',', '.', '!', '?', '-', '~', '…', 'A', 'E', 'I', 'N', 'O', 'Q', 'U', 'a', 'b', 'd', 'e', 'f', 'g', '

In [18]:
pip install opencc-python-reimplemented

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 484 kB 28.5 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486153 sha256=4868d4272078632002bf58f5da00af13f295b745034602b179bff35e632c5b88
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [1]:
import torch
a = torch.Tensor([1000, 1000, 1000]).cuda()

In [41]:
%cd /lib/python3.7/

/lib/python3.7
